# Linelist generator

This is a simple notebook to generate a reduced linelist for use by `cloudy` in the grid creation process. This works by analysing various cloudy models including stellar and AGN models at different metallicity and then combining the line lists where lines are bright enough in at least one model.

In [1]:
import numpy as np
from synthesizer.photoionisation import cloudy23

In [2]:
# define the wavelength type, either standard or just using vacuum wavelengths.
wavelength_type = 'standard'
# wavelength_type = 'vacuum'

# define the reference line to use, by default here we use Hbeta
if wavelength_type == 'standard':
    reference_line = 'H 1 4861.32A'
if wavelength_type == 'vacuum':
    reference_line = 'H 1 4862.69A'

# log constrast relative to Hbeta to consider, i.e. only include lines which are within this value of the reference line.
contrast_stars = 1.5
contrast_agn = 1.0

# minimum wavelength of lines to consider
min_wavelength = 1000

# maximum wavelength of lines to consider
max_wavelength = 25000

### Stellar models

Read in AGN model line lists and identify lines above the threshold.

In [3]:

linelist = []

model = r'metallicity_'+wavelength_type
for i in range(4):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast_stars)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])



FileNotFoundError: metallicity_standard/1.lines not found.

### AGN models

Read in AGN model line lists and identify lines above the threshold.

In [ ]:

model = r'relagn_'+wavelength_type

for i in range(3):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast_agn)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])


In [ ]:
linelist = list(set(linelist))
linelist.sort()

# print(linelist)
print(len(linelist))

linelist_ = []

for line in linelist:

    element, ion, wavelength = line.split(' ')

    if len(ion)==1:

        line_ = f'{element} {ion} {wavelength}'
        print(line_)
        linelist_.append(line_)


with open(f'linelist-{wavelength_type}.dat', 'w') as file:
    file.writelines('\n'.join(linelist_) + '\n')

## Convert between standard and vacuum wavelengths

In [ ]:
def vacuum_to_air(wave):

    wave2 = wave**2.0

    fact = 1.0 + 2.735182e-4 + 131.4182 / wave2 + 2.76249e8 / (wave2**2.0)

    fact = fact * (wave >= 2000.0) + 1.0 * (wave < 2000.0)

    wave = wave / fact

    return wave

def air_to_vacuum(wave):

    sigma2 = (1.0e4 / wave) ** 2.0  # Convert to wavenumber squared

    # Compute conversion factor

    fact = 1.0 + 6.4328e-5 + 2.94981e-2 / (146.0 - sigma2) + 2.5540e-4 / (41.0 - sigma2)

    fact = fact * (wave >= 2000.0) + 1.0 * (wave < 2000.0)

    wave = wave * fact  # Convert Wavelength

    return wave


air_to_vacuum(vacuum_to_air(4365.))

In [ ]:

# check this works correctly

standard = open('linelist-standard.dat', 'r').readlines()
vacuum = open('linelist-vacuum.dat', 'r').readlines()

print(standard)

for standard_, vacuum_ in zip(standard, vacuum):

    # remove escape character
    standard_ = standard_[:-1] 
    vacuum_ = vacuum_[:-1] 

    element, ion, wavelength = standard_.split(' ')

    wavelength_ = float(wavelength[:-1])
    wavelength_unit = wavelength[-1]

    # convert to vacuum
    new_vacuum = f'{element} {ion} {air_to_vacuum(wavelength_):.6g}{wavelength_unit}'

    print(standard_, wavelength_, wavelength_unit, '|', new_vacuum, '|', vacuum_)